In [5]:
import os
import requests
import pandas as pd

In [6]:
csv_file_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/MLCQCodeSmellSamples.csv'  # Replace with your CSV file path
df = pd.read_csv(csv_file_path)
filtered_df = df[df['smell'] == 'feature envy']
filtered_df.to_csv('/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/filtered_dataset_feature_envy.csv', index=False)
num_rows = filtered_df.shape[0]  
print(f"Number of rows in the filtered dataset: {num_rows}")
print("Filtered CSV has been saved as 'filtered_dataset_feature_envy.csv'")

Number of rows in the filtered dataset: 3337
Filtered CSV has been saved as 'filtered_dataset_feature_envy.csv'


In [7]:
def convert_to_raw_url(blob_url):
    if 'github.com' in blob_url and '/blob/' in blob_url:
        raw_url = blob_url.replace('github.com', 'raw.githubusercontent.com').replace('/blob/', '/')
        raw_url = raw_url.split('/#')[0]  
        return raw_url
    else:
        raise ValueError(f"Invalid GitHub blob URL: {blob_url}")

In [8]:
def download_and_extract_lines(raw_url, start_line, end_line, save_path):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(raw_url, headers=headers, allow_redirects=True, timeout=10)
        response.raise_for_status()       
        print(f"Downloading from: {raw_url}")
        print(f"Extracting lines {start_line} to {end_line}")
        if 'text/plain' not in response.headers.get('Content-Type', ''):
            print(f"Unexpected content type: {response.headers.get('Content-Type')}")
            print(f"Response preview: {response.text[:500]}")
            return False

        file_content = response.text.splitlines() 
        if start_line < 1 or end_line > len(file_content) or start_line > end_line:
            raise ValueError(
                f"Invalid start_line ({start_line}) or end_line ({end_line}) for file: {raw_url}"
            )
        extracted_lines = file_content[start_line - 1:end_line]
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        with open(save_path, 'w', encoding='utf-8') as file:
            file.write('\n'.join(extracted_lines))
        print(f"Downloaded and extracted lines {start_line}-{end_line}: {raw_url} -> {save_path}")
        return True

    except requests.exceptions.RequestException as e:
        print(f"HTTP error while accessing {file_url}: {e}")
    except ValueError as ve:
        print(f"Value error: {ve}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    return False

In [9]:
dataset_path = "/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/metrics/dataset"
csv_file_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/filtered_dataset_feature_envy.csv'  
df = pd.read_csv(csv_file_path)
filtered_df = df[df['smell'] == 'feature envy']
os.makedirs(dataset_path, exist_ok=True)

ids = []
classes = []

for _, row in filtered_df.iterrows():
    try:
        file_url = row['link']
        raw_url = convert_to_raw_url(file_url)
        file_id = row['id']
        start_line = row['start_line']
        end_line = row['end_line']
        print(file_url)
        print(raw_url)
        file_name = f"{file_id}.java"  # Save as ID with .java extension
        file_path = os.path.join(dataset_path, file_name)
        download_and_extract_lines(raw_url, start_line, end_line, file_path)

        # Update the lists
        if os.path.exists(file_path):
            ids.append(file_id)
            with open(file_path, "r", encoding="ISO-8859-1") as file:
                classes.append(file.read())
        else:
            print(f"File not found for ID {file_id} at {file_path}")

    except Exception as e:
        print(f"Error processing row with ID {row['id']}: {e}")

print("Files added to the 'metrics' folder.")
print(f"Total files processed: {len(ids)}")

https://github.com/apache/syncope/blob/114c412afbfba24ffb4fbc804e5308a823a16a78/client/idrepo/ui/src/main/java/org/apache/syncope/client/ui/commons/ConnIdSpecialName.java/#L35-L37
https://raw.githubusercontent.com/apache/syncope/114c412afbfba24ffb4fbc804e5308a823a16a78/client/idrepo/ui/src/main/java/org/apache/syncope/client/ui/commons/ConnIdSpecialName.java
Extracting lines 35 to 37
Downloaded and extracted lines 35-37: https://raw.githubusercontent.com/apache/syncope/114c412afbfba24ffb4fbc804e5308a823a16a78/client/idrepo/ui/src/main/java/org/apache/syncope/client/ui/commons/ConnIdSpecialName.java -> /Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/metrics/dataset/526.java
https://github.com/apache/tika/blob/4131c6e30f2e0eb1feb85e0f7576531d4e830468/tika-parsers/src/main/java/org/apache/tika/parser/ocr/TesseractOCRConfig.java/#L531-L534
https://raw.githubusercontent.com/apache/tika/4131c6e30f2e0eb1feb85e0f7576531d4e830468/tika-parsers/src/main/java/org/apache/tika/parser/oc

In [ ]:
# java -jar target/ck-0.7.1-SNAPSHOT-jar-with-dependencies.jar \
# "/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/metrics/dataset" \
# 0 \
# 0 \
# true \
# "/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/metrics"